In [2]:
import quandl
import requests
import pandas as pd
import gzip
import json
import os

quandl.read_key(filename=os.environ.get('QUANDL_API'))

In [170]:
resp = requests.get('https://www.sec.gov/Archives/edgar/full-index/2020/QTR1/company.gz')
data = gzip.decompress(resp.content).decode()
lines = data.split('\n')

In [171]:
lines[8:]

['Company Name                                                  Form Type   CIK         Date Filed  File Name',
 '---------------------------------------------------------------------------------------------------------------------------------------------',
 '&VEST Domestic Fund II LP                                     D           1800903     2020-01-27  edgar/data/1800903/0001800903-20-000001.txt         ',
 '&VEST Offshore Fund II L.P.                                   D           1800902     2020-01-27  edgar/data/1800902/0001800902-20-000001.txt         ',
 '&vest Domestic Fund II KPIV, L.P.                             D           1802417     2020-02-06  edgar/data/1802417/0001802417-20-000001.txt         ',
 '024 Pharma, Inc.                                              8-K/A       1307969     2020-02-20  edgar/data/1307969/0001683168-20-000541.txt         ',
 '1 800 FLOWERS COM INC                                         10-Q        1084869     2020-02-07  edgar/data/1084869/000

In [172]:
import re
multispace = re.compile('\s{2,}')
out = [multispace.split(x) for x in lines[10:]]
df = pandas.DataFrame(out)
df.shape

(238283, 7)

In [173]:
df.head()

,0,1,2,3,4,5,6
0,&VEST Domestic Fund II LP,D,1800903,2020-01-27,edgar/data/1800903/0001800903-20-000001.txt,,None
1,&VEST Offshore Fund II L.P.,D,1800902,2020-01-27,edgar/data/1800902/0001800902-20-000001.txt,,None
2,"&vest Domestic Fund II KPIV, L.P.",D,1802417,2020-02-06,edgar/data/1802417/0001802417-20-000001.txt,,None
3,"024 Pharma, Inc.",8-K/A,1307969,2020-02-20,edgar/data/1307969/0001683168-20-000541.txt,,None
4,1 800 FLOWERS COM INC,10-Q,1084869,2020-02-07,edgar/data/1084869/0001437749-20-002005.txt,,None


In [174]:
pads = ['col_{}'.format(x) for x in range(df.shape[1])]
cols = multispace.split(lines[8])
cols.extend(pads[len(cols):])
cols

['Company Name',
 'Form Type',
 'CIK',
 'Date Filed',
 'File Name',
 'col_5',
 'col_6']

In [175]:
df.columns = cols

In [176]:
all_companies = list(set(df['Company Name']))
len(all_companies)

60806

In [177]:
all_companies

['',
 'COMPASS CAPITAL MANAGEMENT, INC',
 'GLASER JONATHAN M',
 'Hou Joy C.',
 'McMurray Kevin',
 'WOOD LAURI A',
 'Chiang Chen Hsiu-Lien',
 'Keeler Anne B',
 'Stubblefield Michael',
 'Aberdeen Standard Platinum ETF Trust',
 'Green Thumb Industries Inc.',
 'Fine Partners I, L.P.',
 'McKenna Dennis F',
 'COMM 2012-CCRE1 Mortgage Trust',
 'Sunworks, Inc.',
 'VEIR, Inc.',
 'HERITAGE HEDGED EQUITY FUND LTD',
 'THOR INDUSTRIES INC',
 'Tottenham Acquisition I Ltd',
 'MOSIER MICHELLE O',
 'COMM 2014-LC17 Mortgage Trust',
 'Banyard R David',
 "O'Connor Michael J",
 'Pirolli James J',
 'Navitas Credit Corp.',
 'Matthews Alan',
 'Cedar Street Teton Fund, L.P.',
 'HARPER HERSHEL',
 'Breeden John E',
 'Emergent BioSolutions Inc.',
 'Frisch Steven J.',
 'AMERICAN OUTDOOR BRANDS CORP',
 'Newbury Equity Partners V (Cayman) L.P.',
 'JDC-JSC OPPORTUNITY OFFSHORE FUND, LTD.',
 'OXY USA INC',
 'SHOR MICHAEL L',
 'SEPARATE ACCOUNT VA-6',
 'DEERFIELD PARTNERS, L.P.',
 'ABERDEEN AUSTRALIA EQUITY FUND INC',


In [1]:
from collections import namedtuple

Listing = namedtuple('Listing', ['company', 'symbol', 'market'])

class Market:
    def __init__(self, link, co_name, symbol, source):
        self.link = link
        self._data = None
        
    @property
    def listings(self):
        if self._data is None:
            self._data = self._fetch_data()
        return self._data
            
    
    def _fetch_data(self):
        ret = []
        resp = requests.get(self.link)
        blob = resp.content.decode()
        data = json.loads(blob)
        for line in data:
            ret.append(Listing(line[self.co_name], line[self.symbol], self.source))
        return ret
    
    def to_pandas(self):
        return pd.DataFrame(self.listings)


In [331]:
nyse = Market(
    link = 'https://datahub.io/core/nyse-other-listings/r/nyse-listed.json',
    co_name = 'Company Name',
    symbol = 'ACT Symbol',
    source = 'nyse')

nasdaq = Market(
    link = 'https://datahub.io/core/nasdaq-listings/r/nasdaq-listed-symbols.json',
    co_name = 'Company Name',
    symbol = 'Symbol',
    source = 'nasdaq')

In [332]:
nasdaq.to_pandas()

,company,symbol,market
0,iShares MSCI All Country Asia Information Tech...,AAIT,nasdaq
1,"American Airlines Group, Inc.",AAL,nasdaq
2,Atlantic American Corporation,AAME,nasdaq
3,"Applied Optoelectronics, Inc.",AAOI,nasdaq
4,"AAON, Inc.",AAON,nasdaq
...,...,...,...
2962,Zion Oil & Gas Inc,ZN,nasdaq
2963,Zynga Inc.,ZNGA,nasdaq
2964,"ZS Pharma, Inc.",ZSPH,nasdaq
2965,"zulily, inc.",ZU,nasdaq
